<a href="https://colab.research.google.com/github/subhs39/PM-Technical-Literacy/blob/main/01-oauth-2.0/Youtube_Taste_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🛠 How to Run the "Vibe Code" App

To run the YouTube Analyzer yourself, you need to generate your own credentials.

### 1. Get Google OAuth Credentials
1.  Go to the [Google Cloud Console](https://console.cloud.google.com/).
2.  Create a new Project.
3.  Search for **"YouTube Data API v3"** and click **Enable**.
4.  Go to **APIs & Services > Credentials**.
5.  Click **Create Credentials > OAuth Client ID**.
6.  Select **Web Application**.
7.  Add `http://localhost:8080` to **Authorized Redirect URIs**.
8.  Copy your `Client ID` and `Client Secret`.

### 2. Get a Gemini API Key
1.  Go to [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  Click **Create API Key**.
3.  Copy the key string.

### 3. Run the Code
1.  Paste your keys into the "Configuration" cell.
2.  Run all cells!

In [3]:
!pip install -q -U google-generativeai

In [11]:
# ==========================================
# CELL 1: CONFIGURATION & SECRETS
# ==========================================
import requests
import urllib.parse
import json
import matplotlib.pyplot as plt
import google.generativeai as genai

# --- 1. YOUR CREDENTIALS (FILL THESE IN) ---
CLIENT_ID = "PASTE_YOUR_CLIENT_ID_HERE"
CLIENT_SECRET = "PASTE_YOUR_CLIENT_SECRET_HERE"
GEMINI_API_KEY = "PASTE_YOUR_GEMINI_API_KEY_HERE"

# --- 2. GLOBAL SETTINGS ---
REDIRECT_URI = "http://localhost:8080"
SCOPES = "https://www.googleapis.com/auth/youtube.readonly"
AUTH_URL = "https://accounts.google.com/o/oauth2/v2/auth"
TOKEN_URL = "https://oauth2.googleapis.com/token"
YOUTUBE_API_URL = "https://www.googleapis.com/youtube/v3"

# Initialize Gemini
genai.configure(api_key=GEMINI_API_KEY)

print("✅ Configuration Loaded.")

✅ Configuration Loaded.


In [17]:
# ==========================================
# CELL 2: CORE FUNCTIONS
# ==========================================

def fetch_liked_videos(token, max_results=50):
    """Fetches video titles from YouTube API"""
    print(f"\n📺 Fetching your last {max_results} liked videos...")
    headers = {"Authorization": f"Bearer {token}"}
    params = {"part": "snippet", "myRating": "like", "maxResults": max_results}

    try:
        response = requests.get(f"{YOUTUBE_API_URL}/videos", headers=headers, params=params)
        if response.status_code == 401:
            print("❌ Error: Access Token Expired or Invalid.")
            return []

        items = response.json().get("items", [])
        return [f"{item['snippet']['title']} (Channel: {item['snippet']['channelTitle']})" for item in items]
    except Exception as e:
        print(f"❌ API Error: {e}")
        return []

def analyze_with_gemini(video_list):
    """Sends video list to Gemini Flash for vibe analysis"""
    if not video_list: return []
    print(f"\n🤖 Sending {len(video_list)} videos to Gemini Flash...")

    # Using the latest stable model
    model = genai.GenerativeModel('gemini-2.0-flash')

    prompt = f"""
    You are a fun, witty YouTube Taste Analyst. Here is a list of the last {len(video_list)} videos I liked:
    {json.dumps(video_list)}

    Task:
    1. Group these videos into 4-6 distinct 'Vibe Categories' based on topic.
    2. Count how many videos are in each category.
    3. For each category, give it a funny/catchy name (e.g., 'The Messi Stan', 'Coding torture').
    4. Write a short, friendly sentence summarizing why I like this topic.

    CRITICAL: Return ONLY valid JSON in this format, no other text:
    [
        {{"category": "Category Name", "count": 10, "summary": "Description here..."}},
        {{"category": "Category Name", "count": 5, "summary": "Description here..."}}
    ]
    """

    try:
        response = model.generate_content(prompt)
        clean_json = response.text.replace("```json", "").replace("```", "").strip()
        return json.loads(clean_json)
    except Exception as e:
        print(f"❌ Gemini Error: {e}")
        return []

def plot_vibes(vibe_data):
    """Generates a professional Pie Chart and Text Report"""
    if not vibe_data:
        print("No data to plot!")
        return

    labels = [item['category'] for item in vibe_data]
    sizes = [item['count'] for item in vibe_data]

    # --- 1. Aesthetics Configuration ---
    # Modern, professional color palette (hex codes)
    colors = ['#264653', '#2a9d8f', '#e9c46a', '#f4a261', '#e76f51', '#457b9d']
    # Slightly separate slices for clarity
    explode = tuple([0.03] * len(sizes))

    # --- 2. Create Plot ---
    fig, ax = plt.subplots(figsize=(12, 7))

    wedges, texts, autotexts = ax.pie(
        sizes,
        # labels=labels,       # <--- Removed labels from the pie itself
        autopct='%1.1f%%',     # Keep percentages on the pie
        startangle=90,         # Start from the top center
        colors=colors,         # Use new color palette
        explode=explode,       # Add separation
        shadow=True,           # Add subtle depth
        textprops=dict(color="white", weight="bold", size=12), # Style the percentages
        wedgeprops={'edgecolor': 'white', 'linewidth': 1.5}   # Clean white lines between slices
    )

    # --- 3. Styling & Legend ---
    ax.axis('equal')  # Ensures the pie is drawn as a perfect circle
    ax.set_title("Your YouTube Vibe Profile", fontsize=18, fontweight='bold', pad=20, color='#333333')

    # Place legend outside the chart to prevent overlapping
    plt.legend(
        wedges,
        labels,
        title="Vibe Categories",
        loc="center left",
        bbox_to_anchor=(1.0, 0, 0.5, 1), # Positions legend to the right
        fontsize=12,
        frameon=False # Removes the box around the legend for a cleaner look
    )

    plt.tight_layout()
    plt.show()

    # --- 4. Text Report ---
    print("\n📝 GEMINI'S REPORT:")
    print("="*40)
    for item in vibe_data:
        print(f"🔹 {item['category']} ({item['count']} videos)")
        print(f"   \"{item['summary']}\"\n")

print("✅ Functions Loaded.")

✅ Functions Loaded.


In [ ]:
# ==========================================
# CELL 3: INITIAL LOGIN (Run Once)
# ==========================================

# 1. Build the URL
params = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "response_type": "code",
    "scope": SCOPES,
    "access_type": "offline",
    "prompt": "consent"
}
url = f"{AUTH_URL}?{urllib.parse.urlencode(params)}"

print("👇 CLICK THIS TO LOG IN 👇")
print(url)
print("-" * 60)

# 2. Handle the Code
code_input = input("Paste the code from the URL here: ")
code = urllib.parse.unquote(code_input)

# 3. Exchange for Tokens
data = {
    "code": code,
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "redirect_uri": REDIRECT_URI,
    "grant_type": "authorization_code"
}

response = requests.post(TOKEN_URL, data=data)
tokens = response.json()

# --- THE UPGRADE: SAVE TO GLOBAL VARIABLE ---
if "refresh_token" in tokens:
    # We save this to a global variable so Cell 4 can see it
    global Auto_Refresh_Token
    Auto_Refresh_Token = tokens['refresh_token']

    print("\n🎉 SUCCESS! Token received and saved to memory.")
    print(f"Refresh Token: {Auto_Refresh_Token[:15]}... (hidden for safety)")
    print("You can now run Cell 4 immediately!")
else:
    print("\n⚠️ Login worked, but NO Refresh Token returned.")
    print("Check if you have already authorized this app. You might need to revoke access to get a new refresh token.")
    if "access_token" in tokens:
        Auto_Refresh_Token = None # No refresh token, but we might have an access token
        # Optional: You could save the access token here for immediate use

In [ ]:
# ==========================================
# CELL 4: RUN THE APP (Fully Automated)
# ==========================================

# --- INPUTS ---
# Change this number to analyze more/fewer videos!
VIDEO_COUNT = 50

# --- LOGIC ---
def get_new_token(refresh_token):
    data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "refresh_token": refresh_token,
        "grant_type": "refresh_token"
    }
    resp = requests.post(TOKEN_URL, data=data)
    return resp.json().get("access_token")

# 1. Check for the Token
if 'Auto_Refresh_Token' in globals() and Auto_Refresh_Token:
    print("🔄 Found Refresh Token in memory. refreshing Access Token...")
    current_refresh_token = Auto_Refresh_Token
else:
    # Fallback for tomorrow (when memory is wiped)
    print("⚠️ No token in memory (did you restart Colab?).")
    current_refresh_token = input("Paste your saved Refresh Token here: ")

# 2. Get Access Token
access_token = get_new_token(current_refresh_token)

if access_token:
    # 3. Fetch Data & Analyze
    videos = fetch_liked_videos(access_token, max_results=VIDEO_COUNT)

    if videos:
        results = analyze_with_gemini(videos)
        plot_vibes(results)
    else:
        print("No videos found.")
else:
    print("❌ Failed to refresh token.")